In [24]:
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import rasterio

In [ ]:
def load_and_harmonize_data(historical_path, cubesat_path):
    historical_data = gpd.read_file(historical_path)
    cubesat_data = gpd.read_file(cubesat_path)

    historical_data = historical_data.to_crs(epsg=4326)
    cubesat_data = cubesat_data.to_crs(epsg=4326)
    
  

    historical_data['timestamp'] = pd.to_datetime(historical_data['timestamp'])
    cubesat_data['timestamp'] = pd.to_datetime(cubesat_data['timestamp'])

    combined_data = pd.concat([historical_data, cubesat_data], ignore_index=True)

    combined_data = pd.concat([historical_data, cubesat_data], ignore_index=True)
    return combined_data

In [ ]:
def compute_ndvi(data):
     data['NDVI'] = (data['NIR'] - data['Red']) / (data['NIR'] + data['Red'])
     return data


In [ ]:
def add_time_features(data):
    data['month'] = data['timestamp'].dt.month
    data['year'] = data['timestamp'].dt.year
    return data

In [ ]:
def detect_deforestation(data, ndvi_threshold=0.3):
    data['deforestation'] = data['NDVI'] < ndvi_threshold
    return data

In [ ]:
def process_satellite_image(image_path, timestamp):
    with rasterio.open(image_path) as src:
        red = src.read(1)
        nir = src.read(2)
        ndvi = (nir - red) / (nir + red + 1e-10)
        return ndvi, timestamp

In [ ]:
def aggregate_ndvi(ndvi, timestamp, region_id='region_1'):
    mean_ndvi = np.nanmean(ndvi)
    return {'timestamp': timestamp, 'region_id': region_id, 'mean_ndvi': mean_ndvi}

In [ ]:
def create_ndvi_timeseries(image_paths, timestamps):
    time_series_data = []
    for image_path, timestamp in zip(image_paths, timestamps):
        ndvi, ts = process_satellite_image(image_path, timestamp)
        aggregated = aggregate_ndvi(ndvi, ts)
        time_series_data.append(aggregated)
    return pd.DataFrame(time_series_data)

In [ ]:
def harmonize_and_compute_ndvi(image_paths, timestamps, target_resolution, target_crs):
    ndvi_list = []
    for image_path, timestamp in zip(image_paths, timestamps):
        data = load_and_resample_image(image_path, target_resolution, target_crs)
        red, nir = data[0], data[1]
        ndvi = (nir - red) / (nir + red + 1e-10)
        ndvi_list.append({'timestamp': pd.to_datetime(timestamp), 'ndvi': ndvi})
    return ndvi_list

In [ ]:
def merge_image_datasets(dataset1, dataset2):
    all_timestamps = sorted(set(d['timestamp'] for d in dataset1 + dataset2))
    merged_data = []

    for timestamp in all_timestamps:
        images = [d for d in dataset1 + dataset2 if d['timestamp'] == timestamp]
        if images:
            mean_ndvi = np.nanmean([np.nanmean(img['ndvi']) for img in images])
            merged_data.append({'timestamp': timestamp, 'mean_ndvi': mean_ndvi})
    return pd.DataFrame(merged_data)

In [ ]:
def prepare_time_series_data(data, target_col='NDVI', lags=12):
    ts_data = data.copy()
    for lag in range(1, lags + 1):
        ts_data[f'lag_{lag}'] = ts_data[target_col].shift(lag)
    ts_data = ts_data.dropna()
    return ts_data

In [ ]:
def forecast_deforestation(ts_data, target_col='NDVI'):
    X = ts_data[[col for col in ts_data.columns if 'lag_' in col]]
    y = ts_data[target_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')



In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(y_test.values, label='Actual', marker='o')
plt.plot(y_pred, label='Predicted', marker='x')
plt.legend()
plt.title('NDVI Forecasting')
plt.show()

return model

In [ ]:
def validate_with_ground_truth(data, ground_truth_path):
    ground_truth = gpd.read_file(ground_truth_path)
    ground_truth = ground_truth.to_crs(data.crs)

    validation_data = gpd.sjoin(data, ground_truth, how='inner', predicate='intersects')
    validation_accuracy = validation_data['deforestation'].mean()
    print(f'Validation Accuracy: {validation_accuracy}')

    return validation_accuracy

In [ ]:
if __name__ == '__main__':
    historical_path = 'path_to_historical_data.geojson'
    cubesat_path = 'path_to_cubesat_data.geojson'
    ground_truth_path = 'path_to_ground_truth_data.geojson'
    image_paths = ['path_to_image1.tif', 'path_to_image2.tif']
    timestamps = ['2022-01-01', '2022-02-01']
    combined_data = load_and_harmonize_data(historical_path, cubesat_path)
    combined_data = compute_ndvi(combined_data)
    combined_data = add_time_features(combined_data)
    combined_data = detect_deforestation(combined_data)
    ts_data = prepare_time_series_data(combined_data)
    model = forecast_deforestation(ts_data)
    historical_ndvi = harmonize_and_compute_ndvi(image_paths, timestamps, target_resolution=(10, 10), target_crs='EPSG:4326')
    cubesat_ndvi = harmonize_and_compute_ndvi(image_paths, timestamps, target_resolution=(10, 10), target_crs='EPSG:4326')
    merged_ndvi = merge_image_datasets(historical_ndvi, cubesat_ndvi)
    plot_ndvi_trends(pd.DataFrame(merged_ndvi))
    validate_with_ground_truth(combined_data, ground_truth_path)